This file creates the function in charge of turning the HO TDR results into HE TDR results

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import numpy as np
import os
import math
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import itertools
import shutil

In [87]:
TDR_HO_path = r"C:\Users\Diego\GenX\GenX.jl-main\example_systems\11_New_England_200928h\system"
qt_years = 23
length_rep_period = 8736

## Load the data

In [88]:
p_demand = "Demand_data.csv"
p_fuel = "Fuels_data.csv"
p_CF = "Generators_variability.csv"

path_demand = os.path.join(TDR_HO_path, p_demand)
path_fuel = os.path.join(TDR_HO_path, p_fuel)
path_CF = os.path.join(TDR_HO_path, p_CF)

df_demand = pd.read_csv(path_demand)
df_fuel = pd.read_csv(path_fuel)
df_CF = pd.read_csv(path_CF)

In [89]:
rep_period = qt_years

## Add columns corresponding to START_SUBPERIODS and INTERIOR_SUBPERIODS

In [90]:
# CAREFULL WITH INDEXES IN JULIA
df_demand["Rep_Period"] = 0
df_demand["START_SUBPERIODS"] = 0 
df_demand["INTERIOR_SUBPERIODS"] = 0

for i in range(int(rep_period)):
    start_index = i*length_rep_period
    last_index = start_index + length_rep_period
    indexes = range(int(start_index), int(last_index), 1)
    df_demand.loc[indexes, "Rep_Period"] = i+1

start_subperiods_indexes = df_demand['Rep_Period'].drop_duplicates().index.tolist()
interior_subperiod_indexes = df_demand.index.difference(start_subperiods_indexes)

df_demand.loc[start_subperiods_indexes, "START_SUBPERIODS"] = 1
df_demand.loc[interior_subperiod_indexes, "INTERIOR_SUBPERIODS"] = 1

## Combine all the columns on which to apply DBSCAN in 'df_tot'

In [91]:
selected_demand_columns = [col for col in df_demand.columns if (col.startswith("Demand") or col == "Time_Index" or col == "Rep_Period"or col == "START_SUBPERIODS" or col == "INTERIOR_SUBPERIODS") and (col != "Demand_Segment")]
selected_fuel_columns = [col for col in df_fuel.columns if ((col != "Time_Index") and (col!="None"))]
selected_CF_columns = [col for col in df_CF.columns if (col != "Time_Index") and (col!="None")]


df_demand_values = df_demand[selected_demand_columns]
df_fuel_values = df_fuel[selected_fuel_columns].drop(index=0).reset_index().drop(columns=["index"])
df_CF_values = df_CF[selected_CF_columns]

df_tot = pd.concat([df_demand_values, df_fuel_values , df_CF_values], axis=1)
df_tot

,Time_Index,Demand_MW_z1,Rep_Period,START_SUBPERIODS,INTERIOR_SUBPERIODS,NG,nuclear_fuel,NGCCS,natural_gas,natural_gas_ccs,solar_pv,onshore_wind,fixed_offshore_wind,float_offshore_wind,battery,ror_hydro,pumped_hydro,nuclear_CF
0,1,12774.356,1,1,0,8.661,0.66,9.61608,1,1,0.0,0.3849,0.1182,0.1182,1,0.552744,1,1
1,2,12270.879,1,0,1,8.661,0.66,9.61608,1,1,0.0,0.3708,0.1128,0.1128,1,0.552744,1,1
2,3,11954.372,1,0,1,8.661,0.66,9.61608,1,1,0.0,0.4552,0.0904,0.0904,1,0.552744,1,1
3,4,11828.352,1,0,1,8.661,0.66,9.61608,1,1,0.0,0.3908,0.0797,0.0797,1,0.552744,1,1
4,5,11882.091,1,0,1,8.661,0.66,9.61608,1,1,0.0,0.2933,0.0809,0.0809,1,0.552744,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200923,200924,13813.919,23,0,1,8.236,0.66,9.19108,1,1,0.0,0.6194,0.7371,0.7371,1,0.552744,1,1
200924,200925,13333.828,23,0,1,8.236,0.66,9.19108,1,1,0.0,0.5568,0.7407,0.7407,1,0.552744,1,1
200925,200926,12735.534,23,0,1,8.236,0.66,9.19108,1,1,0.0,0.5715,0.7579,0.7579,1,0.552744,1,1
200926,200927,11949.174,23,0,1,8.236,0.66,9.19108,1,1,0.0,0.5797,0.7281,0.7281,1,0.552744,1,1


In [92]:
df_demand_core = df_demand[df_demand.columns.difference(selected_demand_columns)]
df_demand_core["Sub_Weights"].loc[0] = length_rep_period
df_demand_core["Timesteps_per_Rep_Period"].loc[0] = length_rep_period
selected_fuel_columns.append("Time_Index")
df_fuel_core = df_fuel.iloc[:1]
selected_CF_columns.append("Time_Index")

output_dir = r"C:\Users\Diego\GenX\GenX.jl-main\example_systems\11_New_England_200928h"
os.makedirs(output_dir, exist_ok=True)

# Group by "Rep_Period" and save each subset
for rep_period, group in df_tot.groupby("Rep_Period"):
    ## Step 1: Save modified date under the same format as the original df
    #Demand.csv
    group["Time_Index"] = range(1,len(group)+1)
    df_demand_modified = group[selected_demand_columns].reset_index(drop=True).dropna()
    df_demand_eps = pd.concat([df_demand_core, df_demand_modified], axis=1).dropna(how='all')

    #Fuel.csv
    df_fuel_modified = group[selected_fuel_columns]
    df_fuel_modified.index = df_fuel_modified.reset_index(drop=True).index +1
    # df_fuel_eps = pd.concat([df_fuel_core, df_fuel_modified]).drop("None", axis=1)
    df_fuel_eps = pd.concat([df_fuel_core, df_fuel_modified]).dropna(how='all')

    # Generators_variability.csv
    df_CF_eps = group[selected_CF_columns].reset_index(drop=True).dropna(how='all')

    ## Step 2: Create a directory and save the data
    path_eps= os.path.join(output_dir + "_system_yr", f"Rep_Period_{rep_period}.csv")
    os.makedirs(path_eps, exist_ok=True)

    # Save the DataFrame as a CSV file in the directory
    path_demand_eps = os.path.join(path_eps, "Demand_data.csv")
    path_fuel_eps = os.path.join(path_eps, "Fuels_data.csv")
    path_CF_eps = os.path.join(path_eps, "Generators_variability.csv")

    df_demand_eps.to_csv(path_demand_eps, index=False)
    df_fuel_eps.to_csv(path_fuel_eps, index=False)
    df_CF_eps.to_csv(path_CF_eps, index=False)

C:\Users\Diego\AppData\Local\Temp\ipykernel_24184\3150622138.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_demand_core["Sub_Weights"].loc[0] = length_rep_period
C:\Users\Diego\AppData\Local\Temp\ipykernel_24184\3150622138.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_demand_core["Timesteps_per_Rep_Period"].loc[0] = length_rep_period
